In [14]:
load_abis(abi_dir)

{'LandTablesAndChairsContract': [{'type': 'function',
   'name': 'checkPlaceTableOrChair',
   'inputs': [{'name': 'landId', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'itemId', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'x', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'y', 'type': 'uint256', 'internalType': 'uint256'}],
   'outputs': [],
   'stateMutability': 'nonpayable'},
  {'type': 'function',
   'name': 'checkRemoveTableOrChair',
   'inputs': [{'name': 'landId', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'itemId', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'x', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'y', 'type': 'uint256', 'internalType': 'uint256'}],
   'outputs': [],
   'stateMutability': 'nonpayable'},
  {'type': 'event',
   'name': 'Store_SpliceDynamicData',
   'inputs': [{'name': 'tableId',
     'type': 'bytes32',
     'indexed': True,
     'internalType': 'ResourceId'},
  

In [47]:
from web3 import Web3
import os
import json
from pathlib import Path


abi_dir = "./cafecosmos-contracts/abi"

def load_abis(abi_root_dir) -> dict:
    """
    Load all ABI files from the given directory structure.
    Returns a dictionary with contract names as keys and their ABIs as values.
    """
    abis = {}
    root_path = Path(abi_root_dir)
    
    # Walk through all subdirectories
    for contract_dir in root_path.iterdir():
        if contract_dir.is_dir():
            # Look for the .abi.json file in each contract directory
            abi_file = next(contract_dir.glob("*.abi.json"), None)
            if abi_file:
                try:
                    with open(abi_file, 'r') as f:
                        contract_name = contract_dir.name.replace(".sol", "")
                        abis[contract_name] = json.load(f)
                except json.JSONDecodeError as e:
                    print(f"Error loading {abi_file}: {e}")
                except Exception as e:
                    print(f"Error processing {abi_file}: {e}")
    
    return abis


In [3]:
# abis = load_abis(abi_dir)
world_abi = abis["IWorld"]
abi_dir = "./cafecosmos-contracts/abi"
rpc = "https://rpc.garnetchain.com"
world_address = "0x0209FE2b6C1a2B8cE12D8d9D6cf2D78a3Ff324FE"

NameError: name 'abis' is not defined

In [ ]:
from web3 import Web3
import os
import json
from pathlib import Path
from eth_abi import decode
from eth_abi.exceptions import DecodingError

TARGET_SELECTOR = "d2838c64"

def get_error_selector(error_abi):
    """Calculate the selector for an error entry in the ABI"""
    inputs = ','.join(inp['type'] for inp in error_abi.get('inputs', []))
    signature = f"{error_abi['name']}({inputs})"
    selector = Web3.keccak(text=signature)[:4].hex()
    return selector

def decode_error_data(error_abi, error_data):
    """Try to decode error data using the error ABI"""
    try:
        # Remove '0x' and the selector (first 4 bytes)
        data = error_data[10:]
        types = [inp['type'] for inp in error_abi.get('inputs', [])]
        decoded = decode(types, bytes.fromhex(data))
        return decoded
    except DecodingError:
        return None

def load_and_process_abi(file_path, target_data):
    """Load and process a single ABI file"""
    try:
        with open(file_path, 'r') as f:
            abi_data = json.load(f)
            
        # Handle different ABI formats
        if isinstance(abi_data, dict):
            if 'abi' in abi_data:
                abi_data = abi_data['abi']
            elif 'contracts' in abi_data:
                for contract_data in abi_data['contracts'].values():
                    if 'abi' in contract_data:
                        process_abi_entries(contract_data['abi'], file_path, target_data)
                return
        
        process_abi_entries(abi_data, file_path, target_data)
                
    except json.JSONDecodeError as e:
        print(f"Error loading {file_path}: {e}")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

def process_abi_entries(abi, file_path, target_data):
    """Process ABI entries looking for matching error selectors"""
    if not isinstance(abi, list):
        return
        
    error_entries = [entry for entry in abi if entry.get('type') == 'error']
    
    for error in error_entries:
        selector = get_error_selector(error)
        if selector == TARGET_SELECTOR:
            print(f"\nFound matching error in {file_path}:")
            print(f"Error Name: {error['name']}")
            print(f"Signature: {error['name']}({','.join(inp['type'] for inp in error.get('inputs', []))})")
            print("Decoded data:", decode_error_data(error, target_data))
            print("Parameters:")
            for inp in error.get('inputs', []):
                print(f"  - {inp['name']}: {inp['type']}")

def find_abi_files(directory):
    """Recursively find all potential ABI files"""
    for root, _, files in os.walk(directory):
        for file in files:
            if any(file.endswith(ext) for ext in ['.abi.json', '.json']) or 'abi' in file.lower():
                yield os.path.join(root, file)

def main():
    error_data = "0xd2838c64000000000000000000000000e919cc7bea733bf4cf3c60e85305b4eff00593260000000000000000000000000000000000000000000000000000000000000000"
    
    print("Searching for matching error selectors...")
    current_dir = os.getcwd()
    
    for file_path in find_abi_files(current_dir):
        load_and_process_abi(file_path, error_data)

if __name__ == "__main__":
    main()

Searching for matching error selectors...
Error loading /Users/nico/Workspace/mud-aw.py/examples/cafecosmos-contracts/abi/abi_formatter.py: Expecting value: line 1 column 1 (char 0)

Found matching error in /Users/nico/Workspace/mud-aw.py/examples/cafecosmos-contracts/abi/systems/CatalogueSystem.sol/CatalogueSystem.abi.json:
Error Name: AccessByNoOperator
Signature: AccessByNoOperator(address,address)
Decoded data: ('0xe919cc7bea733bf4cf3c60e85305b4eff0059326', '0x0000000000000000000000000000000000000000')
Parameters:
  - owner: address
  - caller: address

Found matching error in /Users/nico/Workspace/mud-aw.py/examples/cafecosmos-contracts/abi/systems/CatalogueSystem.sol/CatalogueSystem.json:
Error Name: AccessByNoOperator
Signature: AccessByNoOperator(address,address)
Decoded data: ('0xe919cc7bea733bf4cf3c60e85305b4eff0059326', '0x0000000000000000000000000000000000000000')
Parameters:
  - owner: address
  - caller: address

Found matching error in /Users/nico/Workspace/mud-aw.py/exa

In [35]:
w3 = Web3(Web3.HTTPProvider(rpc))

In [37]:
worldw3 = w3.eth.contract(address=world_address, abi=world_abi)

In [38]:
world = worldw3.functions

In [44]:
world.calculateLandCost(1, 1).call()

100000000000

In [62]:
world.placeItem(1, 1, 1, 1).call()

ContractCustomError: ('0xd2838c64000000000000000000000000e919cc7bea733bf4cf3c60e85305b4eff00593260000000000000000000000000000000000000000000000000000000000000000', '0xd2838c64000000000000000000000000e919cc7bea733bf4cf3c60e85305b4eff00593260000000000000000000000000000000000000000000000000000000000000000')

In [7]:
from web3 import Web3
import os
import json
from pathlib import Path

def find_abi_files(root_dir):
    """Recursively find all ABI files"""
    abi_files = []
    root_path = Path(root_dir)
    abi_patterns = ["*.abi.json", "*.json"]
    for pattern in abi_patterns:
        abi_files.extend(root_path.rglob(pattern))
    return abi_files

def load_abis(root_dir) -> dict:
    """Load all ABI files from directory structure"""
    abis = {}
    for abi_file in find_abi_files(root_dir):
        try:
            with open(abi_file, 'r') as f:
                abi_data = json.load(f)
                
            if isinstance(abi_data, dict):
                if 'abi' in abi_data:
                    abi_data = abi_data['abi']
                elif 'contracts' in abi_data:
                    for contract_name, contract_data in abi_data['contracts'].items():
                        if 'abi' in contract_data:
                            abis[contract_name] = contract_data['abi']
                    continue
                
            contract_name = abi_file.parent.name if abi_file.name == 'abi.json' else abi_file.stem.replace('.abi', '')
            abis[contract_name] = abi_data
                
        except Exception as e:
            print(f"Error processing {abi_file}: {e}")
    
    return abis

class World:
    def __init__(self, rpc, world_address, abis_dir):
        self.w3 = Web3(Web3.HTTPProvider(rpc))
        self.abis = load_abis(abis_dir)
        
        if "IWorld" in self.abis:
            self.contract = self.w3.eth.contract(address=world_address, abi=self.abis["IWorld"])
            self.errors = self._extract_all_errors()
            
            for func_name in dir(self.contract.functions):
                if not func_name.startswith("_"):
                    original_function = getattr(self.contract.functions, func_name)
                    setattr(self, func_name, self._wrap_function(original_function, func_name))
        else:
            raise Exception("IWorld ABI not found")

    def _extract_all_errors(self):
        """Extract all errors from all loaded ABIs"""
        errors = {}
        for contract_name, abi in self.abis.items():
            if not isinstance(abi, list):
                continue
            
            for item in abi:
                if item.get('type') == 'error':
                    signature = f"{item['name']}({','.join(inp['type'] for inp in item.get('inputs', []))})"
                    selector = self.w3.keccak(text=signature)[:4].hex()
                    errors[selector] = (contract_name, item['name'])
        return errors

    def _wrap_function(self, contract_function, func_name):
        def wrapped_function(*args, **kwargs):
            try:
                return contract_function(*args, **kwargs).call()
            except Exception as e:
                error_str = str(e)
                if '0x' in error_str:
                    import re
                    hex_match = re.search(r'0x[a-fA-F0-9]+', error_str)
                    if hex_match:
                        selector = hex_match.group(0)[2:10]
                        if selector in self.errors:
                            contract, error = self.errors[selector]
                            error_msg = f"{error} in {contract} when calling {func_name}"
                            # Create new error with just the message
                            new_error = type(e)((error_msg,))
                            raise new_error from None
                raise e
        return wrapped_function

In [9]:
abi_dir = "./cafecosmos-contracts/abi"
rpc = "https://rpc.garnetchain.com"
world_address = "0x0209FE2b6C1a2B8cE12D8d9D6cf2D78a3Ff324FE"

world = World(rpc, world_address, abi_dir)
world.placeItem(1, 1, 1, 1)

ContractCustomError: (('AccessByNoOperator in GuildSystem when calling placeItem',), None)

In [12]:
class Player:
    def __init__(self, private_key: str, world: World = None):
        self.private_key = private_key
        self.player_address = self._derive_address(private_key)
        self.world = world

    def add_world(self, world: World, world_name: str = None):
        self.world = world
        self.world.name = world_name
        if world:
            for func_name in dir(world.contract.functions):
                if not func_name.startswith("_"):
                    setattr(self, func_name, self._wrap_world_function(func_name))

    def _derive_address(self, private_key):
        private_key = private_key if private_key.startswith('0x') else '0x' + private_key
        account = Web3().eth.account.from_key(private_key)
        return account.address

    def _wrap_world_function(self, func_name):
        def wrapped_function(*args, **kwargs):
            world_function = getattr(self.world.contract.functions, func_name)
            transaction = world_function(*args, **kwargs).buildTransaction({
                'from': self.player_address,
                'gas': 200000,
                'gasPrice': self.world.w3.eth.gas_price,
                'nonce': self.world.w3.eth.getTransactionCount(self.player_address),
                'chainId': 1,
            })

            try:
                world_function(*args, **kwargs).call({'from': self.player_address})
            except Exception as e:
                raise RuntimeError(f"Simulation failed: {e}")

            signed_txn = self.world.w3.eth.account.sign_transaction(transaction, self.private_key)
            tx_hash = self.world.w3.eth.sendRawTransaction(signed_txn.rawTransaction)
            print(f"Transaction sent. Hash: {self.world.w3.toHex(tx_hash)}")

            receipt = self.world.w3.eth.wait_for_transaction_receipt(tx_hash)
            print(f"Transaction confirmed: {receipt}")
            return receipt

        return wrapped_function


ValueError: The private key must be exactly 32 bytes long, instead of 3 bytes.